In [1]:
import pandas as pd
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from gensim.models import Word2Vec

导入库函数

In [2]:
train_filename = 'E:/tulip-flip/flip01/data/train.json'
test_filename = 'E:/tulip-flip/flip01/data//test.json'

In [3]:
train = pd.read_json(train_filename)

In [4]:
train.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [5]:
label = list(train['cuisine'].unique())

In [6]:
def label_dataset(row):
    num_label = label.index(row)
    return num_label

train['label'] = train['cuisine'].apply(label_dataset)

In [7]:
train.tail()

,cuisine,id,ingredients,label
39769,irish,29109,"[light brown sugar, granulated sugar, butter, ...",16
39770,italian,11462,"[KRAFT Zesty Italian Dressing, purple onion, b...",6
39771,irish,2238,"[eggs, citrus fruit, raisins, sourdough starte...",16
39772,chinese,41882,"[boneless chicken skinless thigh, minced garli...",8
39773,mexican,2362,"[green chile, jalapeno chilies, onions, ground...",7


In [8]:
stop = ['，','。','[',']','{','}','"',':',',']

In [9]:
train['ingredients'] = [[j for j in i if j not in stop]for i in train['ingredients']]

In [10]:
train.tail()

,cuisine,id,ingredients,label
39769,irish,29109,"[light brown sugar, granulated sugar, butter, ...",16
39770,italian,11462,"[KRAFT Zesty Italian Dressing, purple onion, b...",6
39771,irish,2238,"[eggs, citrus fruit, raisins, sourdough starte...",16
39772,chinese,41882,"[boneless chicken skinless thigh, minced garli...",8
39773,mexican,2362,"[green chile, jalapeno chilies, onions, ground...",7


In [11]:
sentences = train['ingredients']

In [12]:
model = Word2Vec(sentences,sg=1,size=300,window=5,min_count=1,negative=3,sample=0.001,hs=1,workers=4)

In [13]:
model.wv.save_word2vec_format("word2vec",binary = "Ture")  #通过该方式保存的模型，能通过文本格式打开，也能通过设置binary是否保存为二进制文件。但该模型在保存时丢弃了树的保存形式（详情参加word2vec构建过程，以类似哈夫曼树的形式保存词），所以在后续不能对模型进行追加训练

In [14]:
model

In [15]:
model.most_similar("green chile",topn=10)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('yellow hominy', 0.7737886905670166),
 ("Campbell's Condensed Cream of Chicken Soup", 0.7605928182601929),
 ('picante sauce', 0.750419020652771),
 ('seasoned ground turkey', 0.7407606840133667),
 ('mexican style 4 cheese blend', 0.7391599416732788),
 ('40% less sodium taco seasoning mix', 0.7374980449676514),
 ('green chile sauce', 0.7366937398910522),
 ('fat-free cheddar cheese', 0.7363815903663635),
 ('40% less sodium taco seasoning', 0.7340618371963501),
 ('green enchilada sauce', 0.7316405773162842)]

In [16]:
type(model)

gensim.models.word2vec.Word2Vec

In [17]:
def getVector_v2(sentences, model):
        vector_list = [ word2vec_model[k] for k in sentences if k in model]
        vector_df = pd.DataFrame(vector_list)
        sentences_vector = vector_df.mean(axis=0).values
        return sentences_vector

In [18]:
import time

In [19]:
print(model)

Word2Vec(vocab=6714, size=300, alpha=0.025)


In [20]:
print(model['green chile'])

[-0.23039174  0.03566215  0.02704169 -0.27687225  0.24211936  0.2314021
  0.11491535  0.11870423  0.14582466 -0.07369235  0.22664581 -0.00996044
  0.07065434 -0.19292407 -0.06428402  0.03325577 -0.02328974  0.03014048
  0.16198252 -0.16053805  0.00676129 -0.25644076  0.08240806 -0.22108832
  0.09631296 -0.01736156 -0.01399462 -0.14135051  0.18267167  0.08712576
  0.17209485 -0.11530011  0.3001061  -0.17491244  0.00289185 -0.08275723
 -0.22543158  0.20476303 -0.08843    -0.01958277 -0.02004959 -0.17079861
  0.04755206  0.18720582  0.05176255  0.29426867  0.01958344  0.21915723
 -0.10980161 -0.145906    0.0113205   0.00424046  0.05865817 -0.03594108
 -0.13094802 -0.1378171  -0.24617018 -0.1268371   0.10502639  0.19400929
  0.15232468  0.04506044  0.14350578 -0.48185802 -0.02818404 -0.01860394
 -0.05000506 -0.30807832 -0.10456243  0.0257093   0.12498315 -0.09828909
  0.10268021 -0.28319594 -0.06833513 -0.14615919 -0.17416708  0.14050812
 -0.10359866 -0.01918209  0.08823033  0.03648512 -0.

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [21]:
print(model['fat-free cheddar cheese'])

[-0.00949457  0.0362183   0.04616025 -0.06445879  0.00230063  0.09969711
  0.05009166 -0.06958866  0.04022865 -0.04552774  0.10851721  0.03622462
  0.09364624 -0.11328992 -0.0606028   0.05956559  0.03359988 -0.0134865
  0.07475218 -0.05680403 -0.09019325 -0.1360806  -0.03348941 -0.05948436
  0.06045002  0.02074012  0.08844453 -0.08111133 -0.04262146  0.04089339
  0.09864538 -0.02565966  0.15906587 -0.05622394 -0.04599289 -0.08676864
 -0.0653168   0.02867664 -0.05663242 -0.08189032  0.01288544  0.00363039
  0.09839349  0.03082085 -0.02366504  0.10944865  0.006965    0.12729828
  0.02736    -0.17798431  0.0443503  -0.03630246 -0.01517211 -0.10538254
 -0.01295827 -0.02533765 -0.12007877 -0.04975316  0.08454844  0.08150944
  0.03796144 -0.04662075 -0.03275313 -0.2495773  -0.15006101  0.08963182
 -0.00826241 -0.05841363 -0.06889158  0.01941449  0.1063216  -0.08864474
 -0.0172949  -0.18736193 -0.03025972 -0.04436461 -0.1701472   0.01596722
 -0.00145079 -0.00435568  0.01996997  0.01062589  0.

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [22]:
model['fat-free cheddar cheese'].shape

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(300,)

In [23]:
type(model['fat-free cheddar cheese'])

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


numpy.ndarray

In [24]:
import time
import pandas as pd
import numpy as np
def getVector_v2(cutWords, model):
        vector_list = [ model[k] for k in cutWords if k in model]
        vector_df = pd.DataFrame(vector_list)
        cutWord_vector = vector_df.mean(axis=0).values
        return cutWord_vector
 
startTime = time.time()
vector_list = []
i = 0
for cutWords in train['ingredients'][:]:
        i += 1
        if i % 1000 ==0:
                print('前%d篇文章形成词向量花费%.2f秒' %(i, time.time()-startTime))
 
        vector_list.append( getVector_v2(cutWords, model) )
X = np.array(vector_list)
print('Total Time You Need To Get X:%.2f秒' % (time.time() - startTime) )

D:\anaconda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
D:\anaconda\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


前1000篇文章形成词向量花费21.07秒
前2000篇文章形成词向量花费39.64秒
前3000篇文章形成词向量花费58.81秒
前4000篇文章形成词向量花费80.47秒
前5000篇文章形成词向量花费101.15秒
前6000篇文章形成词向量花费121.86秒
前7000篇文章形成词向量花费142.98秒
前8000篇文章形成词向量花费165.74秒
前9000篇文章形成词向量花费184.70秒
前10000篇文章形成词向量花费203.36秒
前11000篇文章形成词向量花费222.09秒
前12000篇文章形成词向量花费240.83秒
前13000篇文章形成词向量花费259.80秒
前14000篇文章形成词向量花费277.58秒
前15000篇文章形成词向量花费292.46秒
前16000篇文章形成词向量花费307.06秒
前17000篇文章形成词向量花费321.48秒
前18000篇文章形成词向量花费335.64秒
前19000篇文章形成词向量花费347.93秒
前20000篇文章形成词向量花费359.86秒
前21000篇文章形成词向量花费371.76秒
前22000篇文章形成词向量花费383.70秒
前23000篇文章形成词向量花费395.58秒
前24000篇文章形成词向量花费407.52秒
前25000篇文章形成词向量花费419.56秒
前26000篇文章形成词向量花费431.50秒
前27000篇文章形成词向量花费443.42秒
前28000篇文章形成词向量花费455.31秒
前29000篇文章形成词向量花费467.25秒
前30000篇文章形成词向量花费479.20秒
前31000篇文章形成词向量花费491.43秒
前32000篇文章形成词向量花费503.31秒
前33000篇文章形成词向量花费515.23秒
前34000篇文章形成词向量花费527.19秒
前35000篇文章形成词向量花费539.06秒
前36000篇文章形成词向量花费551.08秒
前37000篇文章形成词向量花费563.03秒
前38000篇文章形成词向量花费574.92秒
前39000篇文章形成词向量花费586.90秒
Total Time You Need To Get X:596.24秒


In [25]:
X

array([[ 0.09724997,  0.09739946, -0.01579126, ...,  0.0686231 ,
         0.11470233, -0.00717194],
       [-0.00900684, -0.01902029, -0.0967633 , ..., -0.03399816,
         0.06904195,  0.03049369],
       [-0.00089942,  0.03514317, -0.1091911 , ...,  0.04595183,
         0.11071311,  0.09322056],
       ...,
       [-0.00846277, -0.02487125, -0.04110322, ..., -0.05495912,
         0.0359963 ,  0.08240642],
       [ 0.03773035, -0.00977357, -0.10278422, ..., -0.01181576,
         0.08347534,  0.17178231],
       [ 0.02006595,  0.14127056, -0.03379859, ...,  0.12411941,
         0.06384745,  0.11633403]])

In [26]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.naive_bayes import MultinomialNB 
from sklearn.model_selection import cross_val_score

In [27]:
train['ingredients'][0]

['romaine lettuce',
 'black olives',
 'grape tomatoes',
 'garlic',
 'pepper',
 'purple onion',
 'seasoning',
 'garbanzo beans',
 'feta cheese crumbles']

In [28]:
model['romaine lettuce']

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.09274483,  0.12376738, -0.1261521 , -0.20065008, -0.00312061,
        0.07870319, -0.0431837 , -0.29415348,  0.14754783,  0.20744376,
        0.0519923 ,  0.05356133,  0.3674951 , -0.08303719,  0.16284953,
        0.04723286,  0.16267332, -0.09941372,  0.06024611,  0.20911066,
       -0.23812857, -0.0669966 ,  0.00060459, -0.2819155 ,  0.26767635,
       -0.20833051,  0.18874931, -0.05059673, -0.13279115,  0.2654979 ,
        0.11455417,  0.1663319 ,  0.405526  , -0.00200046, -0.09359349,
        0.06338648, -0.05019826,  0.11823484, -0.16442074, -0.3601868 ,
       -0.19789569, -0.05062525,  0.39830425,  0.13938725,  0.06264462,
        0.38122073,  0.04412154,  0.09931935,  0.16952063, -0.26840416,
        0.20433898,  0.10618153,  0.01738429, -0.12522414,  0.2079432 ,
       -0.06613404, -0.17287128, -0.20987806,  0.206304  ,  0.4272385 ,
       -0.01248276,  0.05969938,  0.02639608, -0.11509161, -0.28665525,
        0.2130745 ,  0.2237591 ,  0.03054644, -0.06177505, -0.05

In [29]:
X[1]

array([-9.00684411e-03, -1.90202878e-02, -9.67632950e-02, -4.33241516e-02,
        2.68288706e-02,  2.21848602e-01, -1.04838662e-01, -9.41997264e-02,
       -4.70209602e-02,  9.04749927e-02,  8.71539838e-02,  7.91386149e-02,
        2.06625658e-02, -9.04374598e-02,  3.80620946e-02,  9.97894486e-04,
       -3.31123162e-02,  4.14768147e-02,  2.97357474e-02,  5.98751416e-02,
       -6.45601194e-02, -7.70275027e-02, -6.88025364e-03, -1.66933795e-01,
        6.51540235e-04, -7.29148457e-02, -8.92853181e-02, -1.85123155e-01,
        1.71462008e-02, -6.77165765e-02, -1.04265467e-01,  8.52429294e-02,
        6.68578564e-03,  1.21273324e-01,  7.08720417e-02, -9.42903886e-02,
       -3.79898684e-02,  4.58147796e-02,  1.56169938e-02,  2.03089708e-02,
        2.17019194e-02, -4.43085459e-02, -3.54582203e-03, -3.09236468e-02,
        7.07254905e-02,  8.23317987e-02, -1.36310654e-01, -1.77968119e-02,
       -6.40337476e-02, -2.82641854e-02,  5.64633327e-02,  9.14306934e-02,
        8.99874605e-03,  

In [30]:
X[1800]

array([ 1.38864592e-02,  6.85689238e-03,  2.31217281e-02, -1.11336240e-01,
       -6.60384074e-02,  3.95093736e-01,  3.29847156e-02, -1.54986042e-01,
       -3.07915378e-03,  7.11079690e-02,  7.95874614e-02,  4.76563214e-02,
        1.00890227e-01, -1.03238692e-01,  1.02388993e-01,  4.79107923e-02,
        4.07975284e-02,  2.69384135e-02,  2.09157443e-02,  2.68631837e-02,
       -2.19928016e-01,  1.09782428e-03, -1.21701246e-02, -8.89462119e-02,
        7.89842897e-02, -1.04551175e-01,  1.77714922e-01, -8.26616443e-02,
        1.86299901e-02,  1.67024683e-01,  1.92638016e-01,  9.59254480e-02,
        8.50246272e-02,  5.76254701e-02, -3.66296538e-02, -1.17824799e-02,
       -1.42713875e-01,  4.60733215e-02, -2.24648210e-01,  1.04392737e-01,
       -5.67198950e-02, -1.12743156e-01,  1.86816101e-02, -6.95742897e-02,
       -1.85038132e-02,  2.15313927e-02, -8.67233264e-02,  1.11046753e-01,
       -3.51053095e-02, -1.02961908e-01,  1.09588948e-01, -6.64860636e-02,
       -1.06445947e-01, -

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, train['label'], test_size=0.3, random_state=0)
 
logistic_model = LogisticRegression()
logistic_model.fit(train_X, train_y)
logistic_model.score(test_X, test_y)

D:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.7294058493254002

In [33]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.naive_bayes import GaussianNB 
from sklearn.model_selection import cross_val_score
NB_model = GaussianNB()
NB_model.fit(train_X, train_y)
NB_model.score(test_X, test_y)

0.569177909997486

In [34]:
X[1].shape

(300,)

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier()
param = {"n_neighbors": [3,5,10]}
gc = GridSearchCV(knn_model,param_grid=param ,cv=10)
gc.fit(train_X, train_y)
print("在测试集上的准确率：",gc.score(test_X, test_y))
print("选择最好的模型是：",gc.best_estimator_)
print("每个超参数每次交叉验证的结果：",gc.cv_results_)

在测试集上的准确率： 0.739881002262633
选择最好的模型是： KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')
每个超参数每次交叉验证的结果： {'mean_fit_time': array([0.64061992, 0.62814035, 0.64532747]), 'std_fit_time': array([0.02656028, 0.00624692, 0.04531246]), 'mean_score_time': array([29.99573081, 30.32341955, 31.79826422]), 'std_score_time': array([0.83617404, 0.12674316, 0.59661535]), 'param_n_neighbors': masked_array(data=[3, 5, 10],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 10}], 'split0_test_score': array([0.72037236, 0.72932331, 0.7311135 ]), 'split1_test_score': array([0.690681  , 0.70824373, 0.71935484]), 'split2_test_score': array([0.69641577, 0.71326165, 0.72724014]), 'split3_test_score': array([0.71402942, 0.72766416, 0.7334051 ]), 'split4_test_score': arr

In [36]:
from sklearn.ensemble import RandomForestClassifier
param = {"n_estimators":[120,200,300,500,800,1200],"max_depth":[5,8,15,25,30]}
rf = RandomForestClassifier()
gc = GridSearchCV(rf,param_grid=param,cv=2)
gc.fit(train_X, train_y)
print("预测准确率：",gc.score(test_X, test_y))
print("查看选择的参数模型：",gc.best_params_)

预测准确率： 0.738707785133663
查看选择的参数模型： {'max_depth': 30, 'n_estimators': 800}


In [37]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

In [38]:
svc_model = LinearSVC()
svc_model.fit(train_X, train_y)
svc_model.score(test_X, test_y)

0.7361937484287271

In [39]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
param = {"C":[0.1, 1, 10]}
svc_model = LinearSVC()
gc = GridSearchCV(svc_model,param_grid=param,cv=10)
gc.fit(train_X, train_y)
print("预测准确率：",gc.score(test_X, test_y))
print("查看选择的参数模型：",gc.best_params_)

D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
D:\anaconda\lib\site-packages\sklearn\svm\bas

预测准确率： 0.7446576720020113
查看选择的参数模型： {'C': 10}


D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


## 使用卷积神经网络

开始

In [52]:
import pandas as pd  
import jieba  
import jieba.analyse as analyse  
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer  
from tensorflow.keras.preprocessing import sequence  
from tensorflow.keras.models import Sequential  
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, MaxPool1D, Conv1D  
from tensorflow.python.keras.layers.embeddings import Embedding  
from tensorflow.python.keras.utils import multi_gpu_model  
from tensorflow.python.keras.models import load_model  
from tensorflow.python.keras import regularizers  # 正则化  
import matplotlib.pyplot as plt  
import numpy as np  
from tensorflow.python.keras.utils import plot_model  
from sklearn.model_selection import train_test_split  
from tensorflow.python.keras.utils.np_utils import to_categorical  
from sklearn.preprocessing import LabelEncoder  
from tensorflow.python.keras.layers import BatchNormalization